## Finetuning GemmaCasualLLM(2B) with LORA using the DataBricks-Dolly-15k dataset

## Setting up the environment:

#### Using uv:
!uv venv "gemmatest"

#### Install ipykernel for using jypyternotes:
!uv pip install ipykernel

Now our environment "gemmatest" is setup.

To start :

gemmatest/Scripts/activate

In [1]:
print("Hello! Im being printed in the new env 'Gemmatest' ")

Hello! Im being printed in the new env 'Gemmatest' 


# Setting up Gemma

!uv pip install gemma

So some notes here, apparently the standard version of the "pip install gemma" doesnt work, so i would suggest using the "pip install +repolink" method- Also raised this issue in their github - https://github.com/google-deepmind/gemma/issues/252

!uv pip install "gemma[test] @ git+https://github.com/google-deepmind/gemma.git"

Also kauldron also has the same issue:

!uv pip install --upgrade git+https://github.com/google-research/kauldron.git

In [2]:
from gemma import gm

In [3]:
import os
os.environ["KERAS_BACKEND"]= "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

In [4]:
import keras
import keras_nlp

/Users/ashly/MahindraWork/gemmaft/gemmatest/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Lets setup the finetuning dataset


In [5]:
import json
data = []
with open('/Users/ashly/MahindraWork/gemmaft/dataset_unzip/databricks-dolly-15k.jsonl') as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

# Only use 1000 training examples, to keep it fast.
data = data[:1000]

In [1]:
#data 

In [16]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")


In [17]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

## Model 
GemmaCasualLM has been loaded in the Memory

In [10]:
prompt = template.format(
    instruction="Who is the president of India?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Who is the president of India?

Response:
The President of India is the head of the state and the head of the government. The President is elected by the members of the Lok Sabha and the Rajya Sabha. The President is elected for a term of five years. The President is the commander-in-chief of the armed forces. The President is the ex-officio chairman of the Rajya Sabha. The President is the ex-officio chairman of the Union Public Service Commission. The President is the ex-officio chairman of the Central Civil Services Board. The President is the ex-officio chairman of the Central Civil Services Tribunal. The President is the ex-officio chairman of the Central Administrative Tribunal. The President is the ex-officio chairman of the Central Administrative Tribunal. The President is the ex-officio chairman of the Central Administrative Tribunal. The President is the ex-officio chairman of the Central Administrative Tribunal. The President is the ex-officio chairman of the Ce

In [11]:
prompt = template.format(
    instruction="Explain the process of photosynthesis.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis.

Response:
Photosynthesis is the process by which plants use light energy to convert water and carbon dioxide into glucose and oxygen. The process of photosynthesis is carried out by the green pigment chlorophyll in the chloroplasts of plant cells. The chloroplasts are the organelles in plant cells that contain the chlorophyll. The chloroplasts are surrounded by a double membrane. The inner membrane is called the thylakoid membrane. The thylakoid membrane is folded into stacks called grana. The grana are the sites of photosynthesis. The grana are surrounded by a fluid called the stroma. The stroma contains the enzymes that are needed for photosynthesis. The stroma is also the site of the Calvin cycle, which is the process by which glucose is made from carbon dioxide. The Calvin cycle is carried out by the enzyme RuBisCO. The Calvin cycle is also called the dark reaction because it takes place in the absence of light. The Calvin cycle 

This is the standard response from the LLM, lets setup the LORA Adapter for Finetuning.

The LoRA rank determines the dimensionality of the trainable matrices that are added to the original weights of the LLM. 

It controls the expressiveness and precision of the fine-tuning adjustments.

A higher rank means more detailed changes are possible, but also means more trainable parameters. 

A lower rank means less computational overhead, but potentially less precise adaptation.

This tutorial uses a LoRA rank of 4. In practice, begin with a relatively small rank (such as 4, 8, 16). 

This is computationally efficient for experimentation.

In [18]:
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [20]:
#data

In [ ]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

  10/1000 ━━━━━━━━━━━━━━━━━━━━ 10:20:28 38s/step - loss: 0.2527 - sparse_categorical_accuracy: 0.3900

To get better responses from the fine-tuned model, you can experiment with:

Increasing the size of the fine-tuning dataset
Training for more steps (epochs)
Setting a higher LoRA rank
Modifying the hyperparameter values such as learning_rate and weight_decay.

## Gemma3 finetuneing using Gemma Library

Takes too much time to load

In [4]:
model = gm.nn.Gemma3_4B()
params = gm.ckpts.load_params(gm.ckpts.CheckpointPath.GEMMA3_4B_IT)

# Example of multi-turn conversation
sampler = gm.text.ChatSampler(
    model=model,
    params=params,
    multi_turn=True,
)

prompt = """Which of the two images do you prefer?

Image 1: <start_of_image>
Image 2: <start_of_image>

Write your answer as a poem."""
out0 = sampler.chat(prompt, images=[image1, image2])

out1 = sampler.chat('What about the other image ?')

KeyboardInterrupt: 